In [ ]:
from PIL import Image, ImageDraw

import numpy as np
from tqdm import tqdm

In [ ]:
# Load and convert a square image to a circular cropped, grayscale image
img = np.array(Image.open("image.png").convert("L"))

radius = img.shape[0] // 2
yy, xx = np.ogrid[:img.shape[0], :img.shape[1]]
circle = (xx - radius) ** 2 + (yy - radius) ** 2 <= radius ** 2
img[~circle] = 0


Image.fromarray(img)

In [ ]:
n_points = 256

string_img = img.copy()
width, height = img.shape
center = (width // 2, height // 2)

angles = np.linspace(0, 2 * np.pi, n_points, endpoint=False)
points_x = center[0] + radius * np.cos(angles)
points_y = center[1] + radius * np.sin(angles)

connections = []
for i in tqdm(range(n_points)):
    for j in range(i + 1, n_points):
        origin = i
        destination = j
        x1, y1 = points_x[origin], points_y[origin]
        x2, y2 = points_x[destination], points_y[destination]
        num = int(np.hypot(x2 - x1, y2 - y1))
        x = np.linspace(x1, x2, num=num)
        y = np.linspace(y1, y2, num=num)
        x = np.clip(np.round(x).astype(int), 0, width - 1)
        y = np.clip(np.round(y).astype(int), 0, height - 1)

        connections.append({
            "origin": origin,
            "destination": destination,
            "x": x,
            "y": y 
        })

string_img = np.ones_like(img) * 255
string_img[~circle] = 0
count = 0
for connection in connections:
    x = connection["x"]
    y = connection["y"]
    if np.average(img[y, x]) < 127.5:
        string_img[y, x] = 0
        count += 1
print(f"Drew {count}/{len(connections)} lines")
Image.fromarray(string_img)

In [ ]:
string_img = 255 * np.ones(img.shape, dtype=np.uint8)
unthreaded = np.ones(len(connections), dtype=bool)

def average_colors(string_img, connections, unthreaded):
    result = 255 * np.ones(len(connections), dtype=float)
    for i, status in enumerate(unthreaded):
        if status is False: continue

        connection = connections[i]
        x = connection["x"]
        y = connection["y"]
        pixels = string_img[y, x]
        result[i] = np.mean(pixels) # This is the difference it would make to the image
    return result

n_strings = 60
history = np.zeros((n_strings,), dtype=int)
for i in tqdm(range(n_strings)):
    # Find the darkest connection
    new_string = np.argmin(average_colors(string_img, connections, unthreaded))
    
    # Paint it white
    connection = connections[new_string]
    x = connection["x"]
    y = connection["y"]
    string_img[y, x] = 255
    # Mark it as threaded
    history[i] = new_string
    unthreaded[new_string] = False

Image.fromarray(string_img)

In [ ]:
# Final image
string_img = 255*np.ones_like(img)
string_img[~circle] = 0

for i in tqdm(range(n_strings)):
    # Find the darkest connection
    new_string = history[i]
    
    # Paint it black
    connection = connections[new_string]
    x = connection["x"]
    y = connection["y"]
    string_img[y, x] = 0

Image.fromarray(string_img)